<a href="https://colab.research.google.com/github/sebsteinig/analysis-mcglannan-2022/blob/main/pyGPlates_site-reconstructions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pyGPlates site reconstructions
Reconstruct paleolocations of present-day locations across the Phanerozoic. PyGplates allows to use some of the functionality of the GUI GPlates software within python scripts. This allows scripting to automatically process many different locations and/or time periods. Different rotation models can be used and are easily exchangeable. I currently use the PALEOMAP rotation model that is consistent to the Bristol Scotese simulations by Paul Valdes.

## Prelude (only necesseary when running on Google Colab)
If running on Google Colab, execute the following cell to download the repo and install pyGPlates on the virtual machine. 

If running somewhere else, you can execute the whole notebooks and this part will be skipped automatically.

In [1]:
# detect if we are running on colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
# configure environment on Colab
    import google.colab

    # if on Colab, clone the repository to access the data locally
    import os
    repo = "analysis-mcglannan-2022"

    # clone repo if it does not already exist
    if not os.path.exists(repo):
        print('cloning GitHub repository '+repo)
        !git clone https://github.com/sebsteinig/{repo}.git
  
    %cd {repo}
    
    # install pygplates
    !sudo apt install ./bin/pygplates_0.36.0_py36_ubuntu-18.04-amd64.deb
    

## User input
define variables/lists to quickly change inputs to the notebook

In [2]:
# input csv file (in data directory) with label, modern latitude, modern longitude
file_input_sites = 'midcontinent_sites.csv'

# list of ages (in Ma) for which we want to reconstruct paleolocations for the input sites
ages = ['385', '380', '375', '370', '366', '359', '354', '349', '344', '339', '333', '327']




## import pygplates and other packages

In [3]:
import os
import sys
import pandas as pd
import csv

# add pygplates to python path
if IN_COLAB:
    sys.path.insert(0, os.path.abspath('/usr/lib')) # ubuntu VM on colab
else:
    sys.path.insert(0, os.path.abspath('./bin/pygplates_0.36.0_py37_Darwin-x86_64')) # macOS Intel 
import pygplates




## load plate model
List of available models at http://portal.gplates.org/portal/rotation_models/.
Here we are using the 'PALEOMAP PaleoAtlas for GPlates'by Scotese et al. (https://www.earthbyte.org/paleomap-paleoatlas-for-gplates/)

In [4]:
# static polygons are the 'partitioning features'
static_polygons = pygplates.FeatureCollection('PALEOMAP_Global_Plate_Model/PALEOMAP_PlatePolygons.gpml')
# actual rotation model
rotation_model=pygplates.RotationModel('PALEOMAP_Global_Plate_Model/PALEOMAP_PlateModel.rot')

## Main code
3-step process to reconstruct paleolocations:
1. combine input points into feature collection
2. assign plate ids to features
3. reconstruct paleolocations for features

In [5]:
# load point coordinates
df_sites = pd.read_csv('data/' + file_input_sites,sep=',')

for ageCount, age in enumerate(ages):
    
    # Create output directories & all intermediate directories if don't exists
    output_dir = './reconstructions/'+ ages[ageCount] + 'Ma'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print("Directory " , output_dir ,  " Created ")
    else:    
        print("Directory " , output_dir ,  " already exists") 
    
    #### step 1: put the points into a feature collection, using Lat,Lon coordinates from dataframe
    point_features = []
    for index,row in df_sites.iterrows():
        point = pygplates.PointOnSphere(float(row.LAT),float(row.LON))
        point_feature = pygplates.Feature()
        point_feature.set_geometry(point)
        point_features.append(point_feature)

    ### step 2: assign plate ids to features
    # To reconstruct any feature geometries, each feature must have a plate id assigned. If they don't already, 
    # then the pygplates function 'PlatePartitioner' performs this function (analogous to the 'assign plate ids' 
    # menu option in GPlates GUI) 
    partitioned_point_features = pygplates.partition_into_plates(static_polygons, rotation_model, point_features) 

    ### step 3: reconstruct paleolocations for features
    input_file_name = file_input_sites.split('.')[0]

    # Results are saved in two different ways:
    
    # 1. save shape files to disk for later use (e.g. load shapefiles into python script for direct plotting)
    pygplates.reconstruct(partitioned_point_features, rotation_model, output_dir + '/' + input_file_name + '_' + ages[ageCount] +'Ma.shp', float(ages[ageCount]))
    
    # 2. output paleolocations to CSV
    reconstructed_feature_geometries = []
    pygplates.reconstruct(partitioned_point_features, rotation_model, reconstructed_feature_geometries, float(ages[ageCount]))    
    
    # create output CSV file
    with open(output_dir + '/' + input_file_name + '_' + ages[ageCount] +'Ma.csv', mode='w') as output_file:
        output_writer = csv.writer(output_file, delimiter=',')
    
        # write header
        output_writer.writerow(['name', 'modern LAT', 'modern LON', 'reconstructed LAT', 'reconstructed LON', 'age [Ma]'])
    
        for siteCount, reconstructed_feature_geometry in enumerate(reconstructed_feature_geometries):
            paleoLocation = reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon()
            
            # additional output to console for quick check
            print('Paleolocation for ' + df_sites.name[siteCount] + ' at ' + ages[ageCount] + 'Ma (LAT/LON): ' + str(round(paleoLocation[0],1)) + '/'+str(round(paleoLocation[1],1)) )
        
            # write data to CSV
            output_writer.writerow([df_sites.name[siteCount], df_sites.LAT[siteCount], df_sites.LON[siteCount], round(paleoLocation[0],2), round(paleoLocation[1],2), ages[ageCount]])  

Directory  ./reconstructions/385Ma  Created 
Paleolocation for Shaffer-Rohling_Core at 385Ma (LAT/LON): -35.7/-46.7
Paleolocation for I35_Sycamore_South at 385Ma (LAT/LON): -37.4/-47.0
Paleolocation for Kansas_OK_Outcrop at 385Ma (LAT/LON): -37.1/-43.7
Paleolocation for Hamsten_Unit_Core at 385Ma (LAT/LON): -34.6/-46.4
Paleolocation for Bakken at 385Ma (LAT/LON): -18.7/-42.6
Directory  ./reconstructions/380Ma  Created 
Paleolocation for Shaffer-Rohling_Core at 380Ma (LAT/LON): -33.9/-46.5
Paleolocation for I35_Sycamore_South at 380Ma (LAT/LON): -35.7/-46.8
Paleolocation for Kansas_OK_Outcrop at 380Ma (LAT/LON): -35.4/-43.6
Paleolocation for Hamsten_Unit_Core at 380Ma (LAT/LON): -32.9/-46.3
Paleolocation for Bakken at 380Ma (LAT/LON): -17.0/-42.6
Directory  ./reconstructions/375Ma  Created 
Paleolocation for Shaffer-Rohling_Core at 375Ma (LAT/LON): -32.2/-46.4
Paleolocation for I35_Sycamore_South at 375Ma (LAT/LON): -34.0/-46.6
Paleolocation for Kansas_OK_Outcrop at 375Ma (LAT/LON): -33